In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data
import xarray as xr
import h5py
import pickle

# Helpful
import time
import datetime
import itertools
from itertools import product
from tqdm import tqdm
import os

# My Methods
import importlib
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
from src.models.EMOS_global.EMOS_global_load_models import *

2023-05-21 18:33:18.398534: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-21 18:33:18.398554: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

### Goal: Denormalize data

In [3]:
var_names = ["u10", "v10", "t2m", "t850", "z500"]

<IPython.core.display.Javascript object>

#### 1. Load Data

In [4]:
dat_train_proc_norm = ldp.load_data_all_train_proc_norm()
dat_test_proc_norm = ldp.load_data_all_test_proc_norm()

<IPython.core.display.Javascript object>

#### 2. Load Means and Stds

1. Variable Index Positions:<br>
    u10: 0 <br>
    v10: 1 <br>
    t2m: 2 <br>
    t850: 5 <br>
    z500: 14 <br>

In [56]:
global_means = np.load("/mnt/sda/Data2/fourcastnet/data/stats_v0/global_means.npy")
global_stds = np.load("/mnt/sda/Data2/fourcastnet/data/stats_v0/global_stds.npy")

<IPython.core.display.Javascript object>

In [57]:
global_means = global_means.flatten()[[0, 1, 2, 5, 14]]

<IPython.core.display.Javascript object>

In [59]:
global_stds.flatten()[[0, 1, 2, 5, 14]]

array([   5.61282367,    4.77990804,   21.42417831,   15.74041515,
       3439.80138962])

<IPython.core.display.Javascript object>

In [80]:
dat_train_proc_norm[0][list(dat_train_proc_norm[0].data_vars.keys())[0]]

<xarray.DataArray 'u10_train' (forecast_date: 1429, lead_time: 32, lat: 120,
                               lon: 130, mean_std: 2)>
array([[[[[-1.131444e+00,  4.768372e-07],
          ...,
          [-4.690776e-01,  0.000000e+00]],

         ...,

         [[-8.239514e-01,  2.384186e-07],
          ...,
          [ 8.775242e-01,  2.384186e-07]]],


        ...,


        [[[-1.383693e+00,  1.003814e+00],
          ...,
          [ 4.684406e-01,  1.881163e-01]],

         ...,

         [[ 9.909421e-01,  4.445735e-01],
          ...,
          [-5.960758e-01,  6.458046e-01]]]],



       ...,



       [[[[-2.736735e+00,  9.536743e-07],
          ...,
          [ 5.847320e-01,  0.000000e+00]],

         ...,

         [[ 6.252213e-01,  1.788139e-07],
          ...,
          [ 8.558252e-02,  2.980232e-08]]],


        ...,


        [[[-1.342113e+00,  8.861129e-01],
          ...,
          [ 5.992896e-02,  2.500250e-01]],

         ...,

         [[-1.592126e-01,  2.555267e-01],
          ...,
          [ 1.371440e+00,  4.299938e-01]]]]], dtype=float32)
Dimensions without coordinates: forecast_date, lead_time, lat, lon, mean_std

<IPython.core.display.Javascript object>

In [60]:
def denormalize(mean, std, x):
    denormalized = (x * std) + mean
    return denormalized

<IPython.core.display.Javascript object>

In [61]:
def denormalize_std(std, x):
    denormalized_std = x * std
    return denormalized_std

<IPython.core.display.Javascript object>

In [82]:
def main(mean, std, dataset, name):
    """
    Denormalize a Dataset and save it at data directory
Args:
    mean (float): mean value to denormalize with
    std (float): standard deviation used to denormalize with
    dataset(dataset): xarray dataset with var_train and var_truth dataArrays
    name (String): name to be given to dataset
Returns:
    None
    
    
    """
    denormalized_mean = xr.apply_ufunc(
        denormalize,
        mean,
        std,
        dataset[list(dataset.data_vars.keys())[0]].isel(mean_std=0),
    )
    denormalized_std = xr.apply_ufunc(
        denormalize_std,
        std,
        dataset[list(dataset.data_vars.keys())[0]].isel(mean_std=1),
    )
    denormalized_truth = xr.apply_ufunc(
        denormalize, mean, std, dataset[list(dataset.data_vars.keys())[1]]
    )
    denormalized_train = xr.concat(
        [denormalized_mean, denormalized_std], dim="mean_std"
    )
    denormalized_train = denormalized_train.transpose(
        "forecast_date", "lead_time", "lat", "lon", "mean_std"
    )
    denormalized_dataset = xr.Dataset(
        data_vars={"u10_train": denormalized_train, "u10_truth": denormalized_truth,}
    )
    denormalized_dataset.to_netcdf(
        "/Data/Delong_BA_Data/mean_ens_std_denorm/" + name + ".h5", format="NETCDF4"
    )

<IPython.core.display.Javascript object>

In [63]:
denormalized_mean = xr.apply_ufunc(
    denormalize, -6.42, 5.61, dat_train_proc_norm[0].u10_train.isel(mean_std=0)
)

<IPython.core.display.Javascript object>

In [64]:
denormalized_std = xr.apply_ufunc(
    denormalize_std, -5.61, dat_train_proc_norm[0].u10_train.isel(mean_std=1)
)

<IPython.core.display.Javascript object>

In [65]:
denormalized_truth = xr.apply_ufunc(
    denormalize, -6.42, 5.61, dat_train_proc_norm[0].u10_truth
)

<IPython.core.display.Javascript object>

In [73]:
denormalized_train = xr.concat([denormalized_mean, denormalized_std], dim="mean_std")

<IPython.core.display.Javascript object>

In [74]:
denormalized_train = denormalized_train.transpose(
    "forecast_date", "lead_time", "lat", "lon", "mean_std"
)

<IPython.core.display.Javascript object>

In [27]:
dat_train_proc_norm[0].u10_train.values.shape

(1429, 32, 120, 130, 2)

<IPython.core.display.Javascript object>

In [29]:
test = xr.Dataset(
    data_vars={
        "u10_train": dat_train_proc_norm[0].u10_train,
        "u10_truth": denormalized,
    }
)

<IPython.core.display.Javascript object>

In [36]:
test.to_netcdf("/Data/Delong_BA_Data/mean_ens_std_denorm/test.h5", format="NETCDF4")

<IPython.core.display.Javascript object>

In [37]:
test2 = xr.open_dataset("/Data/Delong_BA_Data/mean_ens_std_denorm/test.h5")

<IPython.core.display.Javascript object>